# 1. 합성곱 신경망 소개

In [1]:
## 간단한 컨브넷 만들기
from tensorflow import keras
from tensorflow.keras import layers

inputs = keras.Input(shape=(28, 28, 1))
x = layers.Conv2D(filters=32, kernel_size=3, activation='relu')(inputs)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation='relu')(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation='relu')(x)
x = layers.Flatten()(x)

outputs = layers.Dense(10, activation='softmax')(x)
model = keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
model.summary()

In [ ]:
## MNIST 이미지에 컨브넷 훈련하기
from tensorflow.keras.datasets import mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255
test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255

model.compile(optimizer='rmsprop',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

model.fit(train_images, train_labels, epochs=5, batch_size=64)

In [ ]:
## 컨브넷 평가하기
test_loss, test_acc = model.evaluate(test_images, test_labels)
print('test_acc:', test_acc)

In [ ]:
## 최대 풀링 층이 빠진 잘못된 구조의 컨브넷
inputs = keras.Input(shape=(28, 28, 1))
x = layers.Conv2D(filters=32, kernel_size=3, activation='relu')(inputs)
x = layers.Conv2D(filters=64, kernel_size=3, activation='relu')(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation='relu')(x)
x = layers.Flatten()(x)

outputs = layers.Dense(10, activation='softmax')(x)
model_no_max_pool = keras.Model(inputs=inputs, outputs=outputs)

model_no_max_pool.summary()

# 2. 소규모 데이터셋에서 밑바닥부터 컨브넷 훈련하기

In [ ]:
## Dogs vs Cats 데이터셋을 다운
from google.colab import files
files.upload()
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/ kaggle.json
!kaggle competitions download -c dogs-vs-cats
!unzip -qq dogs-vs-cats.zip
!unzip -qq train.zip

In [ ]:
## 이미지를 훈려, 검증, 테스트 폴더로 복사하기
import os, shutil, pathlib

original_dir = pathlib.Path('./dogs-vs-cats/train') ## 원본 데이터셋을 압축 해제한 디렉터리 경로
new_base_dir = pathlib.Path('./dogs-vs-cats_small') ## 서브셋 데이터셋을 저장할 디렉터리 경로

## start_idx ~ end_idx-1의 고양이 이미지를 new_base_dir/cat(or dog)으로 복사하기 위한 유틸리티 함수
## subset_name: 'train', 'validation', 'test' 중 하나
def make_subset(subset_name, start_idx, end_idx):
    for category in ['cat', 'dog']:
        dir = new_base_dir / subset_name / category
        os.makedirs(dir)
        fnames = [f'{category}.{i}.jpg' for i in range(start_idx, end_idx)]

        for fname in fnames:
            shutil.copyfile(original_dir / fname, dir / fname)

make_subset('train', start_idx=0, end_idx=1000) ## 카테고리마다 처음 1000개의 이미지를 훈련 서브셋으로 복사
make_subset('validation', start_idx=1000, end_idx=1500) ## 카테고리마다 그다음 500개의 이미지를 검증 서브셋으로 복사
make_subset('test', start_idx=1500, end_idx=2000) ## 카테고리마다 그다음 1000개의 이미지를 테스트 서브셋으로 복사

In [ ]:
## 강아지 vs 고양이 이미지 분류 컨브넷 만들기
from tensorflow import keras
from tensorflow.keras import layers

inputs = keras.Input(shape=(180, 180, 3)) ## 이 모델은 180x180 크기의 컬러 이미지 입력을 받음
## 입력 이미지의 픽셀 값을 [0, 1] 사이로 스케일 조정, 각 필터랑 3 채널(RGB)값을 더함
x = layers.Rescaling(1./255)(inputs)
x = layers.Conv2D(filters=32, kernel_size=3, activation='relu')(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation='relu')(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation='relu')(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation='relu')(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation='relu')(x)
x = layers.Flatten()(x)

outputs = layers.Dense(1, activation='sigmoid')(x) ## 이진 분류를 위한 시그모이드 활성화 함수
model = keras.Model(inputs=inputs, outputs=outputs)

## 모델의 구조를 확인
model.summary()

In [ ]:
## 모델 컴파일하기
model.compile(optimizer='rmsprop',
                loss='binary_crossentropy',
                metrics=['accuracy'])

In [ ]:
## image_dataset_from_directory() 함수를 사용하여 디스크에 저장된 이미지 데이터를 읽기
from tensorflow.keras.preprocessing import image_dataset_from_directory

train_dataset = image_dataset_from_directory(
    new_base_dir / 'train',
    image_size=(180, 180),
    batch_size=32)

validation_dataset = image_dataset_from_directory(
    new_base_dir / 'validation',
    image_size=(180, 180),
    batch_size=32)

test_dataset = image_dataset_from_directory(
    new_base_dir / 'test',
    image_size=(180, 180),
    batch_size=32)

In [ ]:
## 데이터셋을 사용하여 모델 훈련하기
callbacks = [ ## 이벤트를 모니터링하고 제어하는 데 사용
    keras.callbacks.ModelCheckpoint( ## 특정 지표(val_loss)가 향상될 때 모델의 가중치를 저장
        filepath='convent_from_scratch.keras', ## 저장할 파일명(경로는 현재 ipynb 파일이 있는 디렉터리)
        save_best_only=True, ## True: 가장 좋은 모델만 저장, False: 모든 모델 저장
        monitor='val_loss',)
]

history = model.fit(
    train_dataset,
    epochs=30,
    validation_data=validation_dataset,
    callbacks=callbacks)

In [ ]:
## 훈련 과정의 정확도와 손실 그래프 그리기
import matplotlib.pyplot as plt

accraucy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(accraucy) + 1)
plt.plot(epochs, accraucy, 'bo', label='Training accraucy')
plt.plot(epochs, val_accuracy, 'b', label='Validation accraucy')
plt.title('Training and validation accraucy')
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
## 테스트 세트에서 모델 평가하기

## 과대적합되기 전의 상태를 평가하기 위해 저장된 파일에서 모델을 로드
test_model = keras.models.load_model('convent_from_scratch.keras')
test_loss, test_acc = test_model.evaluate(test_dataset)
print(f'Test accuracy: {test_acc:3f}')

In [ ]:
## 컨브넷에 추가할 데이터 증식 단계 정의하기
data_augmentation = keras.Sequential(
    [
        ## 랜덤하게 50% 이미지를 수평으로 뒤집음
        layers.RandomFlip('horizontal'),
        ## [-10%, +10%] 범위 안에서 랜덤한 값만큼 입력 이미지를 회진 (전체 원에 대한 비율이고 각도로 나타내면 [-36,36]에 해당)
        layers.RandomRotation(0.1),
        ## [-20%, +20%] 범위 안에서 랜덤한 비율만큼 이미지를 확대 또는 축소
        layers.RandomZoom(0.1),
    ]
)

In [ ]:
import matplotlib.pyplot as plt

## 랜덤하게 증식된 훈련 이미지 출력
plt.figure(figsize=(10, 10))
## 첫 번째 데이터 배치 반환
for images, _ in train_dataset.take(1):
    for i in range(9):
         ## 배치 이미지에 데이터 증식을 적용
        augmented_images = data_augmentation(images)
        ax = plt.subplot(3, 3, i + 1)
        ## 배치 출력에서 첫 번째 이미지 출력
        ## 갘은 이미지를 아홉 번 반복하는 동안 매법 다른 증식 결과가 나타남
        plt.imshow(augmented_images[0].numpy().astype('uint8'))
        plt.axis('off')

In [ ]:
## 이미지 증식과 드롭아웃을 포함한 컨브넷 만들기
inputs = keras.Input(shape=(180, 180, 3))
x = data_augmentation(inputs)
x = layers.Rescaling(1./255)(x)
x = layers.Conv2D(filters=32, kernel_size=3, activation='relu')(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=64, kernel_size=3, activation='relu')(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=128, kernel_size=3, activation='relu')(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation='relu')(x)
x = layers.MaxPooling2D(pool_size=2)(x)
x = layers.Conv2D(filters=256, kernel_size=3, activation='relu')(x)
x = layers.Flatten()(x)
x = layers.Dropout(0.5)(x) ## 드롭아웃 추가

outputs = layers.Dense(1, activation='sigmoid')(x)
model = keras.Model(inputs=inputs, outputs=outputs)

model.compile(optimizer='rmsprop',
                loss='binary_crossentropy',
                metrics=['accuracy'])

In [ ]:
## 규제를 추가한 컨브넷 훈련하기
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath='convnet_from_scratch_with_augmentation.keras',
        save_best_only=True,
        monitor='val_loss')
]

history = model.fit(
    train_dataset,
    epochs=100,
    validation_data=validation_dataset,
    callbacks=callbacks)

In [ ]:
## 훈련 과정의 정확도와 손실 그래프 그리기
import matplotlib.pyplot as plt

accraucy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(accraucy) + 1)
plt.plot(epochs, accraucy, 'bo', label='Training accraucy')
plt.plot(epochs, val_accuracy, 'b', label='Validation accraucy')
plt.title('Training and validation accraucy')
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
## 테스트 세트의 정확도를 확인해보자
test_model = keras.models.load_model("convnet_from_scratch_with_augmentation.keras")
test_loss, test_acc = test_model.evaluate(test_dataset)
print(f"테스트 정확도: {test_acc:.3f}")

In [ ]:
## VGG16 합성곱 기반 층 만들기
conv_base = keras.applications.vgg16.VGG16(
		## 모델을 초기화할 imagenet 가중치
    weights='imagenet',
    ## 밀집 연결 분류기를 포함 여부(true가 기본값)
    ## 강아지와 고양이를 구분하는 밀집 연결 층을 훈련시키기 위해 imagenet 가중치는 불필요
    include_top=False,
    ## 네트워크에 주입할 이미지 텐서의 크기
    ## 이 값을 지정하지 않으면 어떤 크기의 입력도 처리할 수 있음
    input_shape=(180, 180, 3))

In [ ]:
conv_base.summary()

In [ ]:
## VGG16 특성과 해당 레이블 추출
import numpy as np

def get_features_and_labels(dataset):
    all_features = []
    all_labels = []
    for images, labels in dataset:
        preprocessed_images = keras.applications.vgg16.preprocess_input(images)
        features = conv_base(preprocessed_images)
         ## (5, 5, 512) 목록 끝에 요소를 추가, 각 요소는 데이터 세트의 각 이미지 배치에 해당하는 기능 배열
        all_features.append(features)
        all_labels.append(labels)

    ## (samples, 5, 5, 512): 단일 연결된 배열 - 표준화되고 편리한 표현 제공
    return np.concatenate(all_features), np.concatenate(all_labels)

In [ ]:
train_features, train_labels = get_features_and_labels(train_dataset) ## 2000
val_features, val_labels = get_features_and_labels(validation_dataset) ## 1000
test_features, test_labels = get_features_and_labels(test_dataset) ## 2000

train_features.shape

In [ ]:
## 밀집 연결 분류기 정의하고 훈련하기
inputs = keras.Input(shape=(5,5,512))
x = layers.Flatten()(inputs) ## Dense 층에 특성을 주입하기 전에 Flatten 층을 사용
x = layers.Dense(256)(x)
x = layers.Dropout(0.5)(x)

outputs = layers.Dense(1, activation='sigmoid')(x)
model = keras.Model(inputs=inputs, outputs=outputs)

model.compile(optimizer='rmsprop',
                loss='binary_crossentropy',
                metrics=['accuracy'])

callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath='feature_extraction.keras',
        save_best_only=True,
        monitor='val_loss')
]

history = model.fit(
    train_features, train_labels,
    epochs=20,
    validation_data =(val_features, val_labels),
    callbacks=callbacks)

In [ ]:
## 결과를 그래프로 나타내기
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
## VGG16 합성곱 기반 층을 만들고 동결하기
conv_base = keras.applications.vgg16.VGG16(
    weights='imagenet',
    include_top=False)
conv_base.trainable = False

In [ ]:
## 동결하기 전과 후에 훈련 가능한 가중치 리스트 출력하기
conv_base.trainalbe = True
print("합성곱 기반 층을 동결하기 전의 훈련 가능한 가주치의 수:", len(conv_base.trainable_weights))
conv_base.trainable = False
print("합성곱 기반 층을 동결한 후의 훈련 가능한 가주치의 수:", len(conv_base.trainable_weights))

In [ ]:
## 데이터 증식 단계와 밀집 분류기를 합성곱 기반 층에 추가하기
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip('horizontal'),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.1),
    ]
)

inputs = keras.Input(shape=(180, 180, 3))
x = data_augmentation(inputs) ## 데이터 증식을 적용
x = keras.applications.vgg16.preprocess_input(x) ## 입력값의 스케일을 조정
x = conv_base(x) ## 사전 훈련된 컨브넷을 적용
x = layers.Flatten()(x)
x = layers.Dense(256)(x)
x = layers.Dropout(0.5)(x)

outputs = layers.Dense(1, activation='sigmoid')(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer='rmsprop',
                loss='binary_crossentropy',
                metrics=['accuracy'])

callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath='feature_extraction_with_data_augmentation.keras',
        save_best_only=True,
        monitor='val_loss')
]

history = model.fit(
    train_dataset,
    epochs=50,
    validation_data=validation_dataset,
    callbacks=callbacks)

In [ ]:
## 결과를 그래프로 나타내기
import matplotlib.pyplot as plt

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
## 테스트 세트에서 모델 평가하기
test_model = keras.models.load_model(
    'feature_extraction_with_data_augmentation.keras')
test_loss, test_acc = test_model.evaluate(test_dataset)
print(f'테스트 정확도: {test_acc:3f}')

In [ ]:
## 모델 미세 조정하기
model.compile(loss='binary_crossentropy',
                optimizer=keras.optimizers.RMSprop(lr=1e-5),
                metrics=['accuracy'])

callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath='fine_tuning.h5',
        save_best_only=True,
        monitor='val_loss')
]

history = model.fit(
    train_dataset,
    epochs=30,
    validation_data=validation_dataset,
    callbacks=callbacks)

In [ ]:
## 테스트 데이터에서 모델 평가하기
model = keras.models.load_model('fine_tuning.h5')
test_loss, test_acc = model.evaluate(test_dataset)
print(f'테스트 정확도: {test_acc:.3f}')